In [487]:
#import qiskit
from qiskit import QuantumCircuit, assemble, Aer,execute
from qiskit.quantum_info import Statevector
from qiskit.visualization import plot_histogram, plot_bloch_vector,array_to_latex
import numpy as np
import random
from itertools import permutations
import math

In [488]:
IDA=[]#[0,1]
IDB=[]#[1,0]
M=[]#[1,1,1,0]
E=[]
backend=Aer.get_backend('aer_simulator')
for i in range(500):
    M.append(np.random.randint(0,1))  
for i in range(100):
    IDA.append(np.random.randint(0,1)) #generating random ID for Alice
    IDB.append(np.random.randint(0,1)) #generating random ID for Bob


In [489]:
st_a=[]
for i in IDA:
    qc1=QuantumCircuit(1,1)
    qc=QuantumCircuit(1,1)
    if i==0:
        qc1.h(0)
        qc1.measure(0,0)
        job1=backend.run(qc1,shots=1,memory=True)
        v1=job1.result().get_memory()
        for j in v1:
            if int(j)==1:
                qc.x(0)
        st_a.append((Statevector.from_instruction(qc).data).tolist())
    else:
        qc1.h(0)
        qc1.measure(0,0)
        job1=backend.run(qc1,shots=1,memory=True)
        v1=job1.result().get_memory()
        for j in v1:
            if int(j)==0:
                qc.h(0)
            else:
                qc.x(0)
                qc.h(0)
        st_a.append((Statevector.from_instruction(qc).data).tolist())
#print(st_a)

In [490]:
st_m=[]
st_m_sp=[]
for i in M:
    qc1=QuantumCircuit(1,1)
    qc=QuantumCircuit(2,2)
    if i==0:
        qc1.h(0)
        qc1.measure(0,0)
        job1=backend.run(qc1,shots=1,memory=True)
        v1=job1.result().get_memory()
        for j in v1:
            if int(j)==0:
                qc.id(0)
                qc.x(1)
            else:
                qc.x(0)
                qc.id(1)
            st_m.append((Statevector.from_instruction(qc).data).tolist())
            #print(Statevector.from_instruction(qc).data)
        
    else:
        qc1.h(0)
        qc1.measure(0,0)
        job1=backend.run(qc1,shots=1,memory=True)
        v1=job1.result().get_memory()
        for j in v1:
            qc.h(0)
            qc.cx(0,1)
            qc.z(0)
            qc.z(1)
            st_m.append((Statevector.from_instruction(qc).data).tolist())
            Statevector.from_instruction(qc).data

for sp in st_m:
    st_m_sp.append(np.array_split(sp,2))
st_m_flat=[itm for sublist in st_m_sp for itm in sublist]
stm=[l.tolist() for l in st_m_flat]

In [491]:
#permutations
st_m_p=[]
l=[item for item in range(0,len(stm))]
random.shuffle(l)
for i in l:
    st_m_p.append(stm[i])
#print(st_m_p)

In [492]:
#insertion of first set of decoy photons
k=len(IDA)
lmd=int(math.floor(len(st_m_p)/k))
sa_p=[]
it=1
lenmsg=len(st_m_p)
for i in IDB:
    #print(i)
    if i==0:
        ins=int(math.floor(lenmsg/it))-lmd
        st_m_p.insert(ins,st_a[i-1])
        sa_p.append(ins)
    else:
        ins=int(math.floor(lenmsg/it))
        st_m_p.insert(ins,st_a[i-1])
        sa_p.append(ins)
    it=it+1
#print(st_m_p)
#print(len(st_m_p))

In [493]:
#second set of decoy photons
ssdp=[[(1+0j), 0j], [0j, (1+0j)], [(0.7071067811865475+0j), (0.7071067811865475+0j)], [(0.7071067811865475+0j), (-0.7071067811865475+0j)]]

In [494]:
#random insertion of second set of decoy photons
#print(lenmsg)
num=random.randint(1,3)
randp=[]
randbs=[]
for i in range(num):
    randp.append(random.randint(0,lenmsg))
randp.sort()
for i in range(num):
    rs=random.randint(0,3)
    st_m_p.insert(randp[i],ssdp[rs])
    randbs.append(rs)
#print(st_m_p)
#print(randp)
#print(randbs)

In [495]:
#security check

k=0
e=0
scb=[]
for i in randp:
    qcs=QuantumCircuit(1,1)
    int_st=Statevector(st_m_p[i])
    #print(st_m_p[i])
    qcs.initialize(int_st.data,0)
    if randbs[k]<=1:
        qcs.measure(0,0)
        jobs=backend.run(qcs,shots=1,memory=True)
        bs=jobs.result().get_memory()
        if bs==['0']:
            scb.append(0)
        else:
            scb.append(1)
    else:
        qcs.h(0)
        qcs.measure(0,0)
        jobs=backend.run(qcs,shots=1,memory=True)
        bs=jobs.result().get_memory()
        if bs==['0']:
            scb.append(2)
        else:
            scb.append(3)
    
    k=k+1
for i in range(len(scb)):
    if scb[i]==randbs[i]:
        e=e+1
for i in reversed(randp):
    del st_m_p[i]
if e/len(scb)>=0.75:
    print("proceed")
    
else:
    print("terminate")
#print(st_m_p)

proceed


In [496]:
#authentication
a_inf=[]
e=0
it=len(IDB)
#print(len(st_m_p))
for i in reversed(IDB):
    qcb=QuantumCircuit(1,1)
    if i==0:
        ins=int(math.floor(lenmsg/it))-lmd
        if IDA[it-1]==0:
            #print(st_m_p[ins])
            qcb.initialize(st_m_p[ins],0)
            qcb.measure(0,0)
            jobb=backend.run(qcb,shots=100,memory=True)
            bs=jobb.result().get_counts()
            if '0' or '1' in bs.keys():
                    a_inf.append(1)
            
        else:
            #Statevector.evolve(st_m_p[ins-1],qcb)
            #print(st_m_p[ins])
            qcb.initialize(st_m_p[ins],0)
            qcb.h(0)
            qcb.measure(0,0)
            jobb=backend.run(qcb,shots=1,memory=True)
            bs=jobb.result().get_counts()
            print(bs)
            if '0' or '1' in bs.keys():
                    a_inf.append(0)
            
    
    else:
        ins=int(math.floor(lenmsg/it))
        if IDA[it-1]==0:
            #Statevector.evolve(st_m_p[ins],qcb)
            qcb.initialize(st_m_p[ins],0)
            #print(st_m_p[ins])
            qcb.measure(0,0)
            jobb=backend.run(qcb,shots=1,memory=True)
            bs=jobb.result().get_counts()
            if '0' or '1' in bs.keys():
                    a_inf.append(1)
            
    
        else:
            #Statevector.evolve(st_m_p[ins],qcb)
            qcb.initialize(st_m_p[ins],0)
            #print(st_m_p[ins])
            qcb.h(0)
            qcb.measure(0,0)
            jobb=backend.run(qcb,shots=1,memory=True)
            bs=jobb.result().get_counts()
            if '0' or '1' in bs.keys():
                    a_inf.append(0)
    del st_m_p[ins]
    #st_m_p.remove(st_m_p[ins])
    it=it-1
#a_inf.reverse()
#print(a_inf)
#print(IDA)
for i in range(len(a_inf)):
    if a_inf[i]!=IDA[i]:
        e=e+1
if e/len(a_inf)<=0.75:
    print("terminate")
else:
    print("proceed")
#print(st_m_p)

proceed


In [497]:
#decoding
dec_m={}
dm={}
dec_circ=[]
k=0
Ans=[]
for i in st_m_p:
    dec_m[l[k]]=i
    k=k+1
for i in sorted(dec_m):
    dm[i]=dec_m[i]
DM=list(dm.values())
for i in range(int(len(DM)/2)):
    dec_circ.append(DM[2*i]+DM[(2*i)+1])
#print(dec_circ)
for i in (dec_circ):
    qcdec=QuantumCircuit(2,2)
    qcdec.initialize(i,[0,1])
    qcdec.measure(0,0)
    qcdec.measure(1,1)
    jobd=execute(qcdec,backend,shots=1024)
    vd=jobd.result().get_counts()
    re=list(vd.keys())
    a= [char for char in re[0]]
    if a[0]==a[1]:
        Ans.append(1)
    else:
        Ans.append(0)
#print(Ans)

In [498]:
if M==Ans:
    print("Message has been decoded")

Message has been decoded


In [499]:
#print("The Message is")
#print(Ans)